In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tenseal as ts
import torch
from sklearn.metrics import accuracy_score, auc, f1_score
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset

In [74]:
data_dir = "/ssd003/projects/pets/datasets"
all_data = pd.read_csv(f"{data_dir}/mtsamples.csv").drop(["Unnamed: 0","description","keywords","sample_name"], axis=1)
colname='medical_specialty'

In [85]:
def filter_speciality(all_data,colname='medical_specialty',cutoff=0.002):
    """Change this so that we can use raw counts as well. This happens to correspond to >= 10 counts"""
    
    assert colname in all_data.columns.values, f"Please check column name ''{colname}''"
    
    
    count_data=all_data['medical_specialty'].value_counts(normalize=True).to_frame() #get counts as data frame
    count_data = count_data.reset_index()
    count_data.columns  = [colname, 'frequency'] # change column names to work with

    
    filtered_data = all_data.merge(count_data, on=colname, how='inner')
    filtered_data.loc[filtered_data['frequency'] < cutoff, colname] = 'Other' ## low-frequency categories are renamed
    filtered_data.drop('frequency',axis=1,inplace=True)
  
    return filtered_data 

all_data_data=filter_speciality(all_data)

In [64]:
random_seed = 314
colname='medical_specialty'
all_data[colname], _ = pd.factorize(all_data[colname])
#print(all_data[colname].value_counts())


7     1103
37     516
2      372
18     355
12     273
6      259
29     230
3      223
9      166
21     160
5      158
33     108
30      98
22      94
28      90
19      83
23      81
32      75
16      70
17      62
13      53
20      51
14      47
35      29
4       27
36      27
24      23
15      21
10      20
31      19
1       18
26      16
38      14
34      10
11      10
8        9
39       8
25       8
0        7
27       6
Name: medical_specialty, dtype: int64


In [68]:
train_data = all_data.sample(frac=0.7, random_state=random_seed)
valid_data = all_data.loc[~all_data.index.isin(train_data.index)]

train_y = train_data[colname].astype("int")
valid_y = valid_data[colname].astype("int")

train_x = train_data.drop(colname, axis=1)
valid_x = valid_data.drop(colname, axis=1)

#train_x
valid_x

,transcription
2,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ..."
3,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
4,1. The left ventricular cavity size and wall ...
5,"PREOPERATIVE DIAGNOSIS: , Morbid obesity.,POST..."
9,"DESCRIPTION:,1. Normal cardiac chambers size...."
...,...
4985,"HISTORY OF PRESENT ILLNESS:, Ms. A is a 55-ye..."
4990,"EXTERNAL EXAMINATION: , The body is that of a ..."
4991,"EXTERNAL EXAMINATION: , The autopsy is begun a..."
4993,"HISTORY:, A 55-year-old female presents self-..."


In [88]:
import nltk